<a href="https://colab.research.google.com/github/mannsgreg/OPENMM/blob/main/Copy_of_3Dto3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title **Install Conda Colab**
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.1.0-1/Mambaforge-23.1.0-1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:19
🔁 Restarting kernel...


In [ ]:
%%capture
!pip install git+https://github.com/pablo-arantes/biopandas
!mamba install openmmforcefields -c conda-forge -y
!pip install openMiChroM

In [ ]:
from OpenMiChroM.ChromDynamics import MiChroM
from OpenMiChroM.CndbTools import cndbTools
import numpy as np

In [ ]:
sim = MiChroM(temperature=1.0, time_step=0.01)

    ***************************************************************************************     
     **** **** *** *** *** *** *** *** OpenMiChroM-1.0.7 *** *** *** *** *** *** **** ****      

         OpenMiChroM is a Python library for performing chromatin dynamics simulations.         
                            OpenMiChroM uses the OpenMM Python API,                             
                employing the MiChroM (Minimal Chromatin Model) energy function.                
      The chromatin dynamics simulations generate an ensemble of 3D chromosomal structures      
      that are consistent with experimental Hi-C maps, also allows simulations of a single      
                 or multiple chromosome chain using High-Performance Computing                  
                            in different platforms (GPUs and CPUs).                             
         OpenMiChroM documentation is available at https://open-michrom.readthedocs.io          

         OpenMiChroM is desc

In [ ]:
sim.setup(platform="cuda")

In [ ]:
sim.saveFolder('102_ep5sig1')

In [ ]:
chr2 = sim.createSpringSpiral(ChromSeq='/content/chr2_compartments.txt', isRing=False)

In [ ]:
sim.loadStructure(chr2, center=True)

In [ ]:
sim.saveStructure(mode='ndb')

In [ ]:
sim.addFENEBonds(kfb=30.0)
sim.addAngles(ka=2.0)
sim.addRepulsiveSoftCore(Ecut=4.0)

In [ ]:
def AM_bond(sim, pair_list, depth, sigma=1):
    R"""
    Internal function that inits FM bond force.
    """
    bondforceGr = sim.mm.CustomBondForce(f'-depth_w*exp((r-r0)^2/(-2.0*sigma^2))')
    bondforceGr.addGlobalParameter('depth_w', depth)
    bondforceGr.addPerBondParameter('r0')
    bondforceGr.addPerBondParameter('sigma')

    sim.forceDict["AM"] = bondforceGr

    for k  in range(len(pair_list)):
        i, j, ro = pair_list[k]
        sim.forceDict["AM"].addBond(int(i), int(j), [ro,np.sqrt(j-i)*sigma])

In [ ]:
pair_list=np.loadtxt('/content/pairs_chr2_h1.txt')
pair_list[:,2]=np.round(pair_list[:,2],4)
print('Number of constrains:',len(pair_list))

Number of constrains: 436645


In [ ]:
AM_bond(sim,pair_list,5,sigma=1)

In [ ]:
sim.addFlatBottomHarmonic(kr=5*10**-3, n_rad=15.0)

In [ ]:
block = 3*10**2
n_blocks = 2*10**3

In [ ]:
for _ in range(n_blocks):
    sim.runSimBlock(block, increment=False)

Number of exceptions: 4843
adding force  FENEBond 0
adding force  AngleForce 1
Add exclusions for RepulsiveSoftCore force
adding force  RepulsiveSoftCore 2
adding force  AM 3
adding force  FlatBottomHarmonic 4
Positions... 
 loaded!
potential energy is 173.740375
bl=0 pos[1]=[374.3 -19.1 -2.6] dr=0.73 t=3.0ps kin=1.62 pot=173.79 Rg=262.259 SPS=2109 
bl=0 pos[1]=[371.9 -17.7 -0.8] dr=1.55 t=6.0ps kin=2.96 pot=172.00 Rg=260.832 SPS=2828 
bl=0 pos[1]=[368.5 -15.9 1.4] dr=2.19 t=9.0ps kin=4.32 pot=169.38 Rg=258.758 SPS=3900 
bl=0 (i) pos[1]=[364.8 -14.5 3.2] dr=2.64 t=12.0ps kin=5.39 pot=166.22 Rg=256.230 SPS=3881 
bl=0 pos[1]=[364.0 -15.2 3.0] dr=0.84 t=15.0ps kin=1.99 pot=165.66 Rg=255.704 SPS=4096 
bl=0 pos[1]=[362.2 -13.8 3.4] dr=1.54 t=18.0ps kin=3.05 pot=163.99 Rg=254.325 SPS=3565 
bl=0 pos[1]=[359.8 -13.5 5.4] dr=2.12 t=21.0ps kin=4.19 pot=161.59 Rg=252.340 SPS=4061 
bl=0 (i) pos[1]=[356.5 -13.4 6.8] dr=2.54 t=24.0ps kin=5.16 pot=158.70 Rg=249.917 SPS=4114 
bl=0 pos[1]=[356.6 -14.0 

In [ ]:
print(sim.chromRG())
sim.saveStructure(mode='ndb')

10.329771


In [ ]:
sim.saveStructure(mode='gro')
sim.saveStructure(mode='pdb')

In [ ]:
sim.removeFlatBottomHarmonic()

Removed FlatBottomHarmonic from the system!


In [ ]:
sim.forceDict.keys()

dict_keys(['FENEBond', 'AngleForce', 'RepulsiveSoftCore', 'AM'])

In [ ]:
sim.initStorage(filename="traj_chr10")

In [ ]:
block = 5*10**2
n_blocks = 5*10**3

In [ ]:
for ok in range(n_blocks):
    sim.runSimBlock(block, increment=True)
    #sim.saveStructure()
    if (ok) % 5 == 0:
       sim.saveStructure(filename=f"structure_{ok}.pdb",mode='pdb')

Streaming output truncated to the last 5000 lines.
bl=1634 pos[1]=[-5.5 7.0 -9.8] dr=0.89 t=14183.0ps kin=1.51 pot=-427.24 Rg=10.358 SPS=3615 
bl=1635 pos[1]=[-4.7 7.0 -8.9] dr=0.90 t=14188.0ps kin=1.52 pot=-427.22 Rg=10.317 SPS=3612 
bl=1636 pos[1]=[-4.7 6.3 -9.5] dr=0.88 t=14193.0ps kin=1.50 pot=-427.22 Rg=10.314 SPS=3610 
bl=1637 pos[1]=[-4.6 6.9 -9.4] dr=0.89 t=14198.0ps kin=1.52 pot=-427.23 Rg=10.296 SPS=3679 
bl=1638 pos[1]=[-5.1 6.4 -9.7] dr=0.89 t=14203.0ps kin=1.52 pot=-427.25 Rg=10.297 SPS=3635 
bl=1639 pos[1]=[-4.9 6.2 -9.9] dr=0.90 t=14208.0ps kin=1.49 pot=-427.21 Rg=10.289 SPS=3616 
bl=1640 pos[1]=[-4.7 6.0 -10.0] dr=0.88 t=14213.0ps kin=1.51 pot=-427.25 Rg=10.268 SPS=3626 
bl=1641 pos[1]=[-5.3 6.6 -10.2] dr=0.88 t=14218.0ps kin=1.52 pot=-427.25 Rg=10.283 SPS=3639 
bl=1642 pos[1]=[-5.6 7.0 -9.7] dr=0.87 t=14223.0ps kin=1.50 pot=-427.24 Rg=10.306 SPS=3647 
bl=1643 pos[1]=[-5.5 6.4 -9.9] dr=0.90 t=14228.0ps kin=1.49 pot=-427.26 Rg=10.317 SPS=3585 
bl=1644 pos[1]=[-5.8 6.3 -9

In [ ]:
sim.storage[0].close()

In [ ]:
sim.saveStructure(mode='pdb')

In [ ]:
import shutil
shutil.make_archive('/content/102_ep5sig1', 'zip', '/content/102_ep5sig1')


'/content/102_ep5sig1.zip'